In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import pandas as pd
import pymysql

In [30]:
def kurlyparsing(keyword):
    driver = webdriver.Chrome(service=Service('driver/chromedriver'))
    url = 'https://www.kurly.com/search?sword=' + keyword
    driver.get(url)

    time.sleep(7)

    html_full = driver.page_source
    html_src = BeautifulSoup(html_full)
    table_list = html_src.select("div.css-i3aepa.e1c07x4811")

    merch_list = pd.DataFrame()
    name_list = []
    price_list = []
    img_link_list = []
    link_list = []
    o_o_o_list = []

    for merch in table_list:
        name = str(merch.select('div.css-1qsrhd.e1c07x485')).split('\">')[1].split('</div>')[0]
        price = int(str(merch.select('span.sales-price.css-1qe668a.ei5rudb1')).split('\">')[1].split(' 원')[0].replace(',',''))
        img_link = str(merch.select("div.e1c07x4810.css-1qpsnts.e3um3060")[0].select("img")[0]).split('src=\"')[1].split('\"/>')[0]

#         print(name,',', price,',', img_link)
        name_list.append(name)
        price_list.append(price)
        img_link_list.append(img_link)

    for i in range(len(table_list)):
        try:
            xpath = '//*[@id="__next"]/div[3]/div/div[2]/div['+str(i+1)+']/div[2]/div[1]'
            driver.find_element(By.XPATH, xpath).click()

            time.sleep(3)

            html_full = driver.page_source
            html_src = BeautifulSoup(html_full)

            link = driver.current_url
            if '장바구니' in str(html_src.select('span.css-ymwvow.e4nu7ef1')) or '바로구매' in str(html_src.select('span.css-ymwvow.e4nu7ef1')):
                out_of_order = 0
            else:
                out_of_order = 1

            driver.back()
            time.sleep(3)

#             print(link, out_of_order)
            link_list.append(link)
            o_o_o_list.append(out_of_order)
        except:
            continue
            
    merch_list = pd.DataFrame({'name':name_list, 'cost':price_list, 'out_of_stock':o_o_o_list, 'link':link_list, 'img_link':img_link_list})
#     print(len(merch_list))
#     print(merch_list.isnull().values.any())
    
    company_code = 1
    return company_code, merch_list

In [31]:
def ssgparsing(keyword):
    driver = webdriver.Chrome(service=Service('driver/chromedriver'))
    url = 'https://www.ssg.com/search.ssg?target=all&query=' + keyword
    driver.get(url)

    time.sleep(7)

    html_full = driver.page_source
    html_src = BeautifulSoup(html_full)
    table_list = html_src.select("div.cunit_lst_v")[0].select('li.cunit_t232')

    merch_list = pd.DataFrame()
    name_list = []
    price_list = []
    img_link_list = []
    link_list = []
    o_o_o_list = []

    for i in range(12):
        try:
            name = str(table_list[i].select('div.title > a > em')[0]).split('ko\">')[1].split('</em>')[0]
            price = int(str(table_list[i].select('div.opt_price > em')).split('price\">')[1].split("</em>")[0].replace(",",""))
            img_link = str(table_list[i].select("div.thmb > a > img")[0]).split('src="//')[1].split('\" width')[0]

            link = 'ssg.com' + str(table_list[i].select('div.thmb > a')[0]).split('href=\"')[1].split('&amp;siteNo')[0]
            if 'sold_out' in str(table_list[i]):
                out_of_order = 1
            else:
                out_of_order = 0
#             print(name,',', price,',',out_of_order,',',link,',', img_link)

            name_list.append(name)
            price_list.append(price)
            o_o_o_list.append(out_of_order)
            link_list.append(link)
            img_link_list.append(img_link)
        except:
            continue


    merch_list = pd.DataFrame({'name':name_list, 'cost':price_list, 'out_of_stock':o_o_o_list, 'link':link_list, 'img_link':img_link_list})
#     print(len(merch_list))
#     print(merch_list.isnull().values.any())
    company_code = 3
    return company_code, merch_list

In [32]:
def coupangparsing(keyword):
    driver = webdriver.Chrome(service=Service('driver/chromedriver'))
    url = 'https://www.coupang.com/np/search?component=&q='+ keyword +'&channel=user'
    driver.get(url)
    time.sleep(7)

    html_full = driver.page_source
    html_src = BeautifulSoup(html_full)
    table_list = html_src.select('div.search-content.search-content-with-feedback > ul')[0].select("li.search-product")

    merch_list = pd.DataFrame()
    name_list = []
    price_list = []
    img_link_list = []
    link_list = []
    o_o_o_list = []

    for i in range(12):
        try:
            name = str(table_list[i].select('div.name')[0]).split('\">')[1].split('</div>')[0]
            price = int(str(table_list[i].select('strong.price-value')).split('\">')[1].split('</strong>')[0].replace(',',''))
            img_link = str(table_list[i].select('dl.search-product-wrap')[0].select('dt.image')[0]).split('src=\"//')[1].split('.jpg')[0] + '.jpg'
            link = 'https://www.coupang.com/vp/products/' + str(table_list[i]).split(' id=\"')[1].split('\" style')[0]

            if '일시품절' in str(table_list[i]):
                out_of_order = 1
            else:
                out_of_order = 0

            name_list.append(name)
            price_list.append(price)
            o_o_o_list.append(out_of_order)
            link_list.append(link)
            img_link_list.append(img_link)
        except:
            continue
            
    merch_list = pd.DataFrame({'name':name_list, 'cost':price_list, 'out_of_stock':o_o_o_list, 'link':link_list, 'img_link':img_link_list})
    company_code = 4
    
    return company_code, merch_list

In [33]:
def dbupload(company_code, merch_list):
    try:
        conn,cur= None, None
        row='none'
        conn = pymysql.connect(host='--', user='--', password='--', db='--', charset='utf8')
        cur = conn.cursor()

        for i in range(len(merch_list)):
            try:
                sql = "INSERT INTO merchandise (companycode, itemname, cost, outofstock, link, imglink) "
                sql+= "VALUES "
                sql+= '('+str(company_code)+',\"'+str(merch_list.loc[i][0])+'\",'+str(merch_list.loc[i][1])+','
                sql+= str(merch_list.loc[i][2])+',\"'+str(merch_list.loc[i][3])+'\",\"'+str(merch_list.loc[i][4])+'\");'
#                 print(sql)

                cur.execute(sql)
                conn.commit()
            except:
                None

    finally:
        conn.close()
    
    return

# 마켓컬리 파싱


# keyword = ''
# company_code, merch_list = kurlyparsing(keyword)
# dbupload(company_code, merch_list)

# SSG 파싱


# keyword = ''
# company_code, merch_list = ssgparsing(keyword)
# dbupload(company_code, merch_list)

# 쿠팡 파싱


# keyword = ''
# company_code, merch_list = coupangparsing(keyword)
# dbupload(company_code, merch_list)

In [29]:
# 쿠팡 파싱
keyword = '브로콜리'
company_code, merch_list = coupangparsing(keyword)
dbupload(company_code, merch_list)

In [27]:
# 마켓컬리 파싱
keyword = '브로콜리'
company_code, merch_list = kurlyparsing(keyword)
dbupload(company_code, merch_list)

In [28]:
# SSG 파싱
keyword = '브로콜리'
company_code, merch_list = ssgparsing(keyword)
dbupload(company_code, merch_list)